In [18]:
import cv2
#以灰階圖的形式存取圖片
graylena = cv2.imread('lena512color.tiff',cv2.IMREAD_GRAYSCALE)
cv2.imshow('original',graylena)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
import numpy as np
#定義有序抖動矩陣
DitherArray= np.array(
    [[0.513,0.272,0.724,0.483,0.543,0.302,0.694,0.453],
    [0.151,0.755,0.091,0.966,0.181,0.758,0.121,0.936],
    [0.634,0.392,0.574,0.332,0.664,0.423,0.604,0.362],
    [0.060,0.875,0.211,0.815,0.030,0.906,0.241,0.845],
    [0.543,0.302,0.694,0.453,0.513,0.272,0.724,0.483],
    [0.181,0.758,0.121,0.936,0.151,0.755,0.091,0.936],
    [0.664,0.423,0.604,0.362,0.634,0.392,0.574,0.332],
    [0.030,0.906,0.241,0.845,0.060,0.875,0.211,0.815]])*255
#複製圖片存到另一個陣列
ditherimage = graylena-0
#取出圖片大小
x,y=ditherimage.shape
'''
將每一個區塊內的像素點與相對於有序抖動矩陣位值之閥值進行比較，
如果大於則該點為白(255)，反之為黑(0)。
'''
for i in range(0,x,8):
    for j in range(0,y,8):
        ditherimage[i:i+8,j:j+8]= (ditherimage[i:i+8,j:j+8]>DitherArray)*255
cv2.imshow('dither',ditherimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
#定義權重擴散矩陣
errorarray=np.array(
[[0.0,    0.0,     0.0,     0.19040, 0.095230],
 [0.04762,0.095230,0.19040, 0.095230,0.04762 ],
 [0.02381,0.047620,0.095230,0.047620,0.02381]])
#複製圖片存到另一個陣列
errordiffusionimage = graylena-0
#取出圖片大小
height,width=errordiffusionimage.shape
#創造一個上下左右個比原本多2格的陣列，以避免違規存取(超出範圍)
tempimage=np.zeros((height+4,width+4))
tempimage[2:height+2,2:width+2]=errordiffusionimage
'''
依序從影像的左上角至右下角處理每一個像素，
目前像素如大於等於128則該點為白(255)，反之則為黑(0)。
並且將誤差依相對應位置與權重擴散出去
'''
for i in range(2,height+2):
    for j in range(2,width+2):
        #記下圖片原本的值
        old = tempimage[i,j]
        #令像素如大於等於128則該點為白(255)，反之則為黑(0)
        new = (old // 127)*255
        #將圖片原本的值改成量化的結果
        tempimage[i,j] = new
        #計算量化誤差(值離邊界0~255的最短距離)
        E = old -new
        #將量化結果加上量化誤差以進行誤差擴散
        tempimage[i:i+3,j-2:j+3] \
            = tempimage[i:i+3,j-2:j+3] +E*errorarray
#將處理好的結果存回原本的陣列
errordiffusionimage = tempimage[2:height+2,2:width+2]
cv2.imshow('error diffusion',errordiffusionimage)
cv2.imwrite('error diffusion.tiff',errordiffusionimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
#定義擴散順序矩陣與擴散權重矩陣
ClassMatrix=np.array(
[[204,  0,  5, 33, 51, 59, 23,118, 54, 69, 40,160,169,110,168,188],
 [  3,  6, 22, 36, 60, 50, 74,115,140, 82,147,164,171,142,220,214],
 [ 14,  7, 42, 16, 63, 52, 94, 56,133,152,158,177,179,208,222,  1],
 [ 15, 26, 43, 75, 79, 84,148, 81,139,136,166,102,217,219,226,  4],
 [ 17, 39, 72, 92,103,108,150,135,157,193,190,100,223,225,227, 13],
 [ 28,111, 99, 87,116,131,155,112,183,196,181,224,232,228, 12, 21],
 [ 47,120, 91,105,125,132,172,180,184,205,175,233,245,  8, 20, 41],
 [ 76, 65,129,137,165,145,178,194,206,170,229,244,246, 19, 24, 49],
 [ 80, 73,106,138,176,182,174,197,218,235,242,249,247, 18, 48, 68],
 [101,107,134,153,185,163,202,173,231,241,248,253, 44, 88, 70, 45],
 [123,141,149, 61,195,200,221,234,240,243,254, 38, 46, 77,104,109],
 [ 85, 96,156,130,203,215,230,250,251,252,255, 53, 62, 93, 86,117],
 [151,167,189,207,201,216,236,239, 25, 31, 34,113, 83, 95,124,114],
 [144,146,191,209,213,237,238, 29, 32, 55, 64, 97,126, 78,128,159],
 [187,192,198,212,  9, 10, 30, 35, 58, 67, 90, 71,122,127,154,161],
 [199,210,211,  2, 11, 27, 37, 57, 66, 89, 98,121,119,143,162,186]])
ErrorArray=np.array(
[[0.38459,1,0.38459],
[1,0,1],
[0.38459,1,0.38459]])
##複製圖片存到另一個陣列
dotdiffutionimg = graylena -0
#取出圖片的大小
height,width = dotdiffutionimg.shape
#創造一個上下左右個比原本多1格的陣列，以避免違規存取(超出範圍)
tempClassMatrix=np.zeros((16+2,16+2))
tempClassMatrix[1:16+1,1:16+1]=ClassMatrix
#將圖片分成16*16對每個區塊進行處理
for i in range(0,height,16):
        for j in range(0,width,16):
            #將區塊取出並存進另一陣列
            tempimage=np.zeros((16+2,16+2))
            tempimage[1:16+1,1:16+1]=dotdiffutionimg[i:i+16,j:j+16]
            #尋找擴0散順序矩陣內的點依序處理
            for k in range(256):
                #紀錄需處理的點位置
                index = np.where(ClassMatrix == k)
                #記下圖片原本的值
                old = tempimage[1+index[0][0],1+index[1][0]]
                #令像素如大於等於128則該點為白(255)，反之則為黑(0)
                new = (old // 127)*255
                #將圖片原本的值改成量化的結果
                tempimage[1+index[0][0],1+index[1][0]] = new
                #計算量化誤差(值離邊界0~255的最短距離)
                E = old -new
                sum = 0
                temperrorarray = np.empty_like(ErrorArray)
                temperrorarray[:] = ErrorArray
                #檢查附近是否有已處理過之點，有則將其權重設為零
                for x in range(-1,2):
                    for y in range(-1,2):
                        if tempClassMatrix[1+y+index[0][0],1+x+index[1][0]] <= k:
                            temperrorarray[y+1,x+1] = 0
                        sum+= temperrorarray[y+1,x+1]
                #計算新的擴散權重矩陣
                if sum != 0:
                    temperrorarray = temperrorarray/sum
                #將量化結果加上權重過後的誤差以進行誤差擴散
                tempimage[1+index[0][0]-1:1+index[0][0]+2,1+index[1][0]-1:1+index[1][0]+2]=tempimage[1+index[0][0]-1:1+index[0][0]+2,1+index[1][0]-1:1+index[1][0]+2]+E*temperrorarray
            #將處理好的區塊存回原陣列
            dotdiffutionimg[i:i+16,j:j+16] = tempimage[1:16+1,1:16+1]
cv2.imshow('dot diffusion',dotdiffutionimg)
cv2.waitKey(0)
cv2.destroyAllWindows()